In [8]:
import cv2
from PIL import Image
from backbones.ghost_model import GhostNet
import torch
import os
import numpy as np
from tqdm import tqdm
from face_alignment import align

from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
# import keras_cv_attention_models
import GhostFaceNets, GhostFaceNets_with_Bias
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [9]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

model = tf.keras.models.load_model('./models/GN_W1.3_S2_ArcFace_epoch48.h5')


In [10]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']

In [11]:
African_wom = os.listdir('../images/test/data/African/Woman')
Caucasian_wom = os.listdir('../images/test/data/Caucasian/Woman')
Asian_wom = os.listdir('../images/test/data/Asian/Woman')
Indian_wom = os.listdir('../images/test/data/Indian/Woman')
Asian_man = os.listdir('../images/test/data/Asian/Man')
African_man = os.listdir('../images/test/data/African/Man')
Caucasian_man = os.listdir('../images/test/data/Caucasian/Man')
Indian_man = os.listdir('../images/test/data/Indian/Man')

In [12]:
def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    brg_img = ((np_img/ 255.) - 0.5) / 0.5
    tensor = tf.convert_to_tensor(np.array([brg_img.transpose(2,0,1)]))    
    return tensor

In [13]:
cache = {}

In [14]:
for race in races:
    landmark = {}
    with open(f'../images/test/txts/{race}/{race}_lmk.txt') as f:
        landmark_lines = f.readlines()
    for line in landmark_lines:
        l = line.strip().split()
        landmark[l[0]] = [float(k) for k in l[2:]]
    with open(f"../images/test/txts/{race}/{race}_pairs.txt", 'r') as f:
        pairs = f.readlines()
    pairs = [pair.strip().split() for pair in pairs]
    for pair in tqdm(pairs):
        try:
            if pair[0] in eval(f'{race}_wom'):
                    gender = "Woman"
            else:
                    gender = "Man"
            sim = 0
            img1 = f"../images/test/data/Just_Images/{race}/{gender}/{pair[0]}_000{pair[1]}.jpg"
            if len(pair) == 3:
                img2 = f"../images/test/data/Just_Images/{race}/{gender}/{pair[0]}_000{pair[2]}.jpg"
                sim = 1
            else:
                if pair[2] in African_wom or pair[2] in Caucasian_wom or pair[2] in Asian_wom or pair[2] in Indian_wom:
                    gen2 = "Woman"
                    if pair[2] in African_wom:
                        race2 = "African"
                    elif pair[2] in Caucasian_wom:
                        race2 = "Caucasian"
                    elif pair[2] in Asian_wom:
                        race2 = "Asian"
                    else:
                        race2 = "Indian"
                else:
                    gen2 = "Man"
                    if pair[2] in African_man:
                        race2 = "African"
                    elif pair[2] in Caucasian_man:
                        race2 = "Caucasian"
                    elif pair[2] in Asian_man:
                        race2 = "Asian"
                    else:
                        race2 = "Indian"
                img2 = f"../images/test/data/Just_Images/{race2}/{gen2}/{pair[2]}_000{pair[3]}.jpg"
            if img1 in cache:
                 vec1 = cache[img1]
            else:
                with torch.no_grad():
                    vec1 = model(np.array(to_input(align.get_aligned_face(img1))).transpose(0,2,3,1))
                vec1/=np.linalg.norm(vec1)
                cache[img1] = vec1
            if img2 in cache:
                vec2 = cache[img2]
            else:
                with torch.no_grad():
                    
                    vec2 = model(np.array(to_input(align.get_aligned_face(img2))).transpose(0,2,3,1))          
                vec2/=np.linalg.norm(vec2)
                cache[img2] = vec2
            if img1 not in cache:
                    cache[img1] = vec1
            if img2 not in cache:
                    cache[img2] = vec2
            score = np.dot(vec1, vec2.T)[0][0]
            with open(f"./{race}_{gender}.csv", 'a') as f:
                f.write(f"{img1},{img2},{score},{sim}\n")
        except Exception as e:
            print(pair, e)
            continue

 25%|██▌       | 1521/6000 [03:36<09:09,  8.16it/s]

['m.02r7h1g', '2', 'm.0bk56n', '5'] [Errno 2] No such file or directory: '../images/test/data/Just_Images/African/Man/m.0bk56n_0005.jpg'


100%|██████████| 6000/6000 [12:08<00:00,  8.24it/s]
